In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('../data/neurips/train_data/train_task_1_2.csv')

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15867850 entries, 0 to 15867849
Data columns (total 6 columns):
 #   Column         Dtype
---  ------         -----
 0   QuestionId     int64
 1   UserId         int64
 2   AnswerId       int64
 3   IsCorrect      int64
 4   CorrectAnswer  int64
 5   AnswerValue    int64
dtypes: int64(6)
memory usage: 726.4 MB


In [4]:
df.head()

,QuestionId,UserId,AnswerId,IsCorrect,CorrectAnswer,AnswerValue
0,16997,65967,12453206,0,4,2
1,16531,62121,15686710,1,1,1
2,15911,50013,13598796,0,3,1
3,1701,104909,10511925,0,4,3
4,22896,21748,941747,0,1,4


In [5]:
meta = {}
for field in {'student', 'question', 'answer'}:
    meta[field] = pd.read_csv(f'../data/neurips/metadata/{field}_metadata_task_1_2.csv')

In [7]:
meta['student'].head()

,UserId,Gender,DateOfBirth,PremiumPupil
0,58022,2,2006-08-01 00:00:00.000,1.0
1,104674,0,NaN,NaN
2,32020,2,2008-11-01 00:00:00.000,1.0
3,81780,2,2004-07-01 00:00:00.000,NaN
4,99967,0,NaN,NaN


In [8]:
meta['question']

,QuestionId,SubjectId
0,13090,"[3, 32, 71, 77, 141, 185, 186, 214]"
1,1855,"[3, 71, 75, 86, 178]"
2,10423,"[3, 32, 38, 239]"
3,2290,"[3, 32, 33, 144]"
4,12785,"[3, 32, 33, 144]"
...,...,...
27608,8083,"[3, 32, 144, 208]"
27609,24671,"[3, 32, 144, 208]"
27610,18085,"[3, 32, 144, 208]"
27611,25102,"[3, 32, 38, 239]"


In [9]:
meta['answer']

,AnswerId,DateAnswered,Confidence,GroupId,QuizId,SchemeOfWorkId
0,11808339.0,2020-03-17 07:55:00.000,NaN,4186,14854,NaN
1,98649.0,2018-12-18 14:23:00.000,NaN,9427,16895,28237.0
2,259238.0,2019-02-21 12:41:00.000,NaN,7651,1127,8386.0
3,17027648.0,2020-03-01 18:13:00.000,NaN,8719,5126,40960.0
4,6579101.0,2019-03-22 14:57:00.000,NaN,9020,13445,NaN
...,...,...,...,...,...,...
19834815,6955999.0,2019-09-27 11:05:00.000,100.0,5092,9378,8375.0
19834816,950591.0,2020-04-29 08:39:00.000,NaN,4924,10639,NaN
19834817,5181829.0,2018-11-16 11:40:00.000,NaN,633,11950,8464.0
19834818,8813030.0,2020-03-07 20:20:00.000,NaN,7176,13776,NaN


In [10]:
meta['answer'].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19834820 entries, 0 to 19834819
Data columns (total 6 columns):
 #   Column          Dtype  
---  ------          -----  
 0   AnswerId        float64
 1   DateAnswered    object 
 2   Confidence      float64
 3   GroupId         int64  
 4   QuizId          int64  
 5   SchemeOfWorkId  float64
dtypes: float64(3), int64(2), object(1)
memory usage: 908.0+ MB


In [12]:
%%time
bundle = (df
    .merge(meta['student'], on='UserId')
    .merge(meta['question'], on='QuestionId')
    .merge(meta['answer'], on='AnswerId')
)

CPU times: user 34.8 s, sys: 2.37 s, total: 37.2 s
Wall time: 37.1 s


In [13]:
bundle.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 15867850 entries, 0 to 15867849
Data columns (total 15 columns):
 #   Column          Dtype  
---  ------          -----  
 0   QuestionId      int64  
 1   UserId          int64  
 2   AnswerId        int64  
 3   IsCorrect       int64  
 4   CorrectAnswer   int64  
 5   AnswerValue     int64  
 6   Gender          int64  
 7   DateOfBirth     object 
 8   PremiumPupil    float64
 9   SubjectId       object 
 10  DateAnswered    object 
 11  Confidence      float64
 12  GroupId         int64  
 13  QuizId          int64  
 14  SchemeOfWorkId  float64
dtypes: float64(3), int64(9), object(3)
memory usage: 1.9+ GB


In [147]:
%%time
bundle['Subjects'] = bundle['SubjectId'].map(lambda x: ' '.join(
    [f's{n.strip()}' for n in x[1:-1].split(',') if level_of[int(n.strip())] < 3]
))

CPU times: user 19.7 s, sys: 375 ms, total: 20.1 s
Wall time: 20.1 s


In [148]:
bundle.head()

,QuestionId,UserId,AnswerId,IsCorrect,CorrectAnswer,AnswerValue,Gender,DateOfBirth,PremiumPupil,SubjectId,DateAnswered,Confidence,GroupId,QuizId,SchemeOfWorkId,Subjects
0,16997,65967,12453206,0,4,2,1,2002-11-01 00:00:00.000,0.0,"[3, 101, 115, 342]",2019-04-24 13:42:00.000,NaN,5026,15391,8408.0,s3 s101 s342
1,16997,86212,15895376,1,4,4,1,2003-09-01 00:00:00.000,0.0,"[3, 101, 115, 342]",2019-07-18 08:18:00.000,NaN,2159,15391,8413.0,s3 s101 s342
2,16997,72019,8030559,1,4,4,0,NaN,NaN,"[3, 101, 115, 342]",2019-03-18 10:42:00.000,NaN,8857,15391,8413.0,s3 s101 s342
3,16997,17397,9602029,1,4,4,1,2005-07-01 00:00:00.000,0.0,"[3, 101, 115, 342]",2019-05-08 15:26:00.000,100.0,3833,15391,8409.0,s3 s101 s342
4,16997,51718,2564614,1,4,4,2,2005-11-01 00:00:00.000,NaN,"[3, 101, 115, 342]",2018-11-18 18:28:00.000,NaN,1369,15391,NaN,s3 s101 s342


In [149]:
df_train.head()

,QuestionId,UserId,AnswerId,IsCorrect,CorrectAnswer,AnswerValue,Gender,DateOfBirth,PremiumPupil,SubjectId,DateAnswered,Confidence,GroupId,QuizId,SchemeOfWorkId,Subjects
5661924,13390,93595,18314688,1,2,2,2,2008-01-01 00:00:00.000,0.0,"[3, 49, 62, 251]",2019-11-07 11:20:00.000,NaN,5445,9609,8386.0,s3 s49 s62 s251
11154109,12522,83216,16491455,1,1,1,2,2009-05-01 00:00:00.000,0.0,"[3, 32, 141, 213]",2019-06-05 10:49:00.000,NaN,3508,9443,8378.0,s3 s32 s141 s213
3323991,13523,57022,4342554,0,4,3,1,2006-03-01 00:00:00.000,0.0,"[3, 32, 36, 223]",2019-12-10 13:11:00.000,NaN,3323,10437,NaN,s3 s32 s36 s223
10374297,14259,117808,8138288,0,2,1,0,NaN,NaN,"[3, 32, 141, 216]",2019-10-19 10:38:00.000,NaN,10792,3668,NaN,s3 s32 s141 s216
6917140,5474,72745,48525,0,4,2,2,2005-04-01 00:00:00.000,NaN,"[3, 32, 34, 141]",2019-06-11 19:43:00.000,NaN,220,5903,NaN,s3 s32 s34 s141


In [21]:
bundle['PremiumPupil'].value_counts()

0.0    4622095
1.0    1341487
Name: PremiumPupil, dtype: int64

In [75]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import LogisticRegression

In [118]:
ohe = OneHotEncoder()
vec = CountVectorizer()
col = ColumnTransformer([('subject', CountVectorizer(), 'Subjects')], remainder=OneHotEncoder())
FIELD = ['UserId', 'Subjects']
display(df_train[FIELD].head())
col.fit_transform(df_train[FIELD].head().fillna(0))#.todense()

,UserId,Subjects
5661924,93595,s3 s49 s62 s251
11154109,83216,s3 s32 s141 s213
3323991,57022,s3 s32 s36 s223
10374297,117808,s3 s32 s141 s216
6917140,72745,s3 s32 s34 s141


array([[0., 0., 0., 0., 1., 1., 0., 0., 0., 1., 1., 0., 0., 0., 1., 0.],
       [1., 1., 0., 0., 0., 1., 1., 0., 0., 0., 0., 0., 0., 1., 0., 0.],
       [0., 0., 0., 1., 0., 1., 1., 0., 1., 0., 0., 1., 0., 0., 0., 0.],
       [1., 0., 1., 0., 0., 1., 1., 0., 0., 0., 0., 0., 0., 0., 0., 1.],
       [1., 0., 0., 0., 0., 1., 1., 1., 0., 0., 0., 0., 1., 0., 0., 0.]])

In [116]:
col.get_feature_names()

['subject__s141',
 'subject__s213',
 'subject__s216',
 'subject__s223',
 'subject__s251',
 'subject__s3',
 'subject__s32',
 'subject__s34',
 'subject__s36',
 'subject__s49',
 'subject__s62',
 'remainder__x0_57022',
 'remainder__x0_72745',
 'remainder__x0_83216',
 'remainder__x0_93595',
 'remainder__x0_117808']

In [150]:
estimators = [
    ('onehot', ColumnTransformer([('subject', CountVectorizer(), 'Subjects')], remainder=OneHotEncoder())),
    ('lr', LogisticRegression())
]
pipe = Pipeline(estimators)

In [58]:
bundle['IsCorrect'].value_counts()

1    10202239
0     5665611
Name: IsCorrect, dtype: int64

In [110]:
%%time
from sklearn.model_selection import train_test_split

df_train, df_test = train_test_split(bundle, shuffle=True, test_size=0.1)

CPU times: user 17 s, sys: 369 ms, total: 17.3 s
Wall time: 17.3 s


In [59]:
len(df_train), len(df_test)

(14281065, 1586785)

In [120]:
FIELDS = (#['UserId', 'QuestionId', 'Gender', 'PremiumPupil', 'GroupId', 'QuizId', 'SchemeOfWorkId']
    ['UserId', 'QuestionId', 'Subjects']
)

In [151]:
%%time
pipe.fit(df_train[FIELDS].fillna(0), df_train['IsCorrect'])

/home/jj/code/ktm/venv/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


CPU times: user 3min 13s, sys: 2min 48s, total: 6min 2s
Wall time: 2min 37s


Pipeline(steps=[('onehot',
                 ColumnTransformer(remainder=OneHotEncoder(),
                                   transformers=[('subject', CountVectorizer(),
                                                  'Subjects')])),
                ('lr', LogisticRegression())])

In [152]:
%%time
pipe.score(df_test[FIELDS].fillna(0), df_test['IsCorrect'])

CPU times: user 6.37 s, sys: 77.5 ms, total: 6.45 s
Wall time: 6.45 s


0.7012304754582379

In [53]:
FORBIDDEN_GROUP_VALUES = [11729]
FORBIDDEN_QUIZ_VALUES = [88, 119, 585, 675, 723, 1341, 1428, 1572, 1764, 2220, 3294, 3314, 3395, 3831, 3947, 4216, 4960, 5277, 5640, 6145, 6437, 6800, 7498, 8065, 8358, 8434, 8930, 9518, 10198, 10247, 10568, 10748, 11143, 11338, 11453, 11704, 11962, 11973, 12157, 13303, 13550, 14122, 14868, 15411, 15615, 16159, 16652, 16847, 16871, 17195]

df_this_test = df_test[FIELDS].replace(
    {'GroupId': {value: None for value in FORBIDDEN_GROUP_VALUES},
     'QuizId': {value: None for value in FORBIDDEN_QUIZ_VALUES}}
)

In [54]:
%%time
pipe.score(df_this_test.fillna(0), df_test['IsCorrect'])

CPU times: user 1.58 s, sys: 40.4 ms, total: 1.62 s
Wall time: 1.62 s


0.6945887439067044

In [39]:
to_search = df_this_test['QuizId'].tolist()
for value in [60, 88, 841, 1047, 1497, 1752, 1764, 2536, 2620, 2955, 3173, 3371, 3395, 3696, 4368, 4519, 4960, 5178, 5686, 5736, 5781, 5897, 6437, 6676, 6838, 6909, 6926, 7059, 7564, 8209, 8624, 10198, 10316, 10431, 10725, 11122, 11222, 11224, 11237, 11363, 11367, 11570, 11735, 11827, 12291, 12313, 12898, 13135, 13182, 13578, 14070, 14998, 15031, 15330, 15397, 15483, 15665, 16302, 16370, 16462, 16679, 16847, 17099, 17228]:
    print(value, value in to_search)

60 True
88 True
841 True
1047 True
1497 True
1752 True
1764 True
2536 True
2620 True
2955 True
3173 True
3371 True
3395 True
3696 True
4368 True
4519 True
4960 True
5178 True
5686 True
5736 True
5781 True
5897 True
6437 True
6676 True
6838 True
6909 True
6926 True
7059 True
7564 True
8209 True
8624 True
10198 True
10316 True
10431 True
10725 True
11122 True
11222 True
11224 True
11237 True
11363 True
11367 True
11570 True
11735 True
11827 True
12291 True
12313 True
12898 True
13135 True
13182 True
13578 True
14070 True
14998 True
15031 True
15330 True
15397 True
15483 True
15665 True
16302 True
16370 True
16462 True
16679 True
16847 True
17099 True
17228 True


In [123]:
sub = pd.read_csv('../data/neurips/metadata/subject_metadata.csv')
sub

,SubjectId,Name,ParentId,Level
0,3,Maths,NaN,0
1,32,Number,3.0,1
2,33,BIDMAS,144.0,3
3,34,Upper and Lower Bounds,141.0,3
4,35,Calculator Use,32.0,2
...,...,...,...,...
383,1982,Mixed operation Fractions,39.0,3
384,1983,Drawing Axes,54.0,3
385,1985,Reading Scales,98.0,3
386,1987,Sorting Shapes,174.0,3


In [146]:
# all_subjects = sub[['SubjectId', 'Name']].set_index('SubjectId').to_dict()['Name']
level_of = sub[['SubjectId', 'Level']].set_index('SubjectId').to_dict()['Level']
level_of

{3: 0,
 32: 1,
 33: 3,
 34: 3,
 35: 2,
 36: 2,
 37: 2,
 38: 2,
 39: 2,
 40: 2,
 41: 3,
 42: 2,
 44: 2,
 45: 2,
 46: 2,
 47: 3,
 48: 2,
 49: 1,
 50: 3,
 51: 3,
 52: 3,
 53: 3,
 54: 2,
 55: 2,
 56: 3,
 57: 3,
 58: 3,
 59: 2,
 60: 3,
 61: 2,
 62: 2,
 63: 3,
 64: 3,
 65: 3,
 66: 3,
 67: 3,
 68: 2,
 69: 3,
 70: 3,
 71: 1,
 72: 3,
 73: 3,
 74: 2,
 75: 3,
 76: 3,
 77: 2,
 78: 3,
 79: 2,
 80: 3,
 81: 2,
 83: 2,
 84: 3,
 85: 3,
 86: 3,
 87: 3,
 88: 3,
 89: 3,
 90: 3,
 91: 2,
 92: 3,
 93: 3,
 94: 3,
 95: 3,
 96: 3,
 97: 2,
 98: 2,
 99: 2,
 100: 2,
 101: 1,
 102: 3,
 103: 3,
 104: 3,
 105: 3,
 106: 3,
 107: 3,
 108: 3,
 109: 3,
 110: 3,
 111: 3,
 112: 3,
 113: 2,
 114: 3,
 115: 3,
 116: 3,
 117: 3,
 118: 3,
 119: 1,
 120: 3,
 126: 2,
 128: 2,
 129: 2,
 130: 2,
 131: 3,
 137: 2,
 139: 2,
 140: 2,
 141: 2,
 142: 2,
 144: 2,
 146: 2,
 149: 1,
 151: 1,
 152: 2,
 153: 2,
 154: 2,
 156: 2,
 157: 3,
 158: 2,
 159: 3,
 160: 3,
 163: 3,
 164: 3,
 165: 3,
 166: 3,
 167: 3,
 168: 3,
 171: 3,
 172: 3,
 173: 

In [145]:
for _, row in list(sub.dropna().iterrows()):
    print(f'"{all_subjects[row.ParentId]}" -> "{row.Name}"')

"Maths" -> "Number"
"Basic Arithmetic" -> "BIDMAS"
"Rounding and Estimating" -> "Upper and Lower Bounds"
"Number" -> "Calculator Use"
"Number" -> "Decimals"
"Number" -> "Factors, Multiples and Primes"
"Number" -> "Fractions, Decimals and Percentage Equivalence"
"Number" -> "Fractions"
"Number" -> "Indices, Powers and Roots"
"Money" -> "Basic Money"
"Number" -> "Negative Numbers"
"Number" -> "Percentages"
"Number" -> "Proportion"
"Number" -> "Ratio"
"Indices, Powers and Roots" -> "Standard Form"
"Number" -> "Surds"
"Maths" -> "Algebra"
"Expanding Brackets" -> "Expanding Single Brackets"
"Expanding Brackets" -> "Expanding Double Brackets"
"Factorising" -> "Factorising into a Single Bracket"
"Factorising" -> "Factorising into a Double Bracket"
"Algebra" -> "Straight Line Graphs"
"Algebra" -> "Quadratic Graphs"
"Other Graphs" -> "Graphs of Trigonometric Functions"
"Other Graphs" -> "Cubics and Reciprocals"
"Other Graphs" -> "Real Life Graphs"
"Algebra" -> "Inequalities"
"Formula" -> "Rearr